<h1>Primo Progetto<h1>

In [1]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json

In [2]:
GoogleScholarSearch.SERP_API_KEY = "5ad7613cf550e6717a98fe89a1313fd076c01bf67a9ff629de82129283fd92cf"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [13]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

,name,affiliations
0,David La Barbera,Università Degli Studi Di Udine
1,Michael Soprano,Postdoctoral Research Fellow at the University...
2,Kevin Roitero,University of Udine
3,Stefano Mizzaro,Full professor of Computer Science and Informa...
4,Damiano Spina,"School of Computing Technologies, RMIT University"
5,Gianluca Demartini,Associate Professor at the University of Queen...
6,Eddy Maddalena,"Università degli Studi di Udine, Italy"


In [14]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
    author_affliation = row['affiliations']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors":f'{author_name},{author_affliation}'
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()['profiles']
    
    for profile in results:
        author_id = profile['author_id']
        cited_by = profile['cited_by']
        
        interests = []
        
        for interest in profile.get('interests', ):
            title = interest.get('title', '')
            interests.append(title)
    
    interests_str = ', '.join(interests)
    
    # Aggiungo una nuova riga al dataframe updated_df
    new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests_str}
    updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

new_df = pd.merge(df, updated_df, left_index=True, right_index=True)
display(new_df)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"Computer Science, Crowdsourcing"
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"Artificial Intelligence, Crowdsourcing and Hum..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"Information retrieval, mobile systems, scholar..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"Information Retrieval, Text Analytics, Data Sc..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"Information Retrieval, Semantic Web, Human Com..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."


In [15]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
new_df.to_csv('data/nodes.csv', index=False)

#### 2 | Per ogni autore utilizzo il suo ID per accedere al relativo profilo GoogleScholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI

In [16]:
df = pd.read_csv('data/nodes.csv')

# Creo le liste vuote per memorizzare i nomi dei co autori e successivamente le loro informazioni
co_authors = []
info_co_authors = []

coauthors_df = pd.DataFrame(columns=['name', 'affiliations', 'author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_id = row['author_id']
    author_name = row['name']    
        
    params = {
        "engine" : "google_scholar_author",
        "hl": "en",
        "mauthors": author_id
    }
    
    search = GoogleScholarSearch(params)
    results = search.get_dict()['co_authors']
    print(results)
    
    for co_author in results:
        co_authors.append(
            {
                "author" : author_name,
                "co_author" : co_author['name']
            }
        )
        
    
    for co_author in results:
        info_co_authors.append(
            {
                "co_author" : co_author['author_id'],
                "co_author" : co_author['name'],
                "affiliations": co_author['affiliations']
            }
        )
    
   



KeyError: 'co_authors'